## 소벨(Sobel) 필터
* OpenCV에서 제공하는 소벨 마스크를 이용한 **영상 미분 함수** (**[1, 2, 1]**)
* **dst = cv2.Sobel(src, ddepth, dx, dy)**
    * dst : src를 편미분한 결과
    * ddepth : dst 영상의 **자료형** (**-1** : src와 같은 타입의 영상 생성)
    * dx, dy : x방향, y방향으로의 편미분 차수 (대부분 1차 미분)

## 샤르(Scharr) 필터
* 3x3 소벨 마스크보다 정확한 미분 계산 수행 (**[3, 10, 3]**)
* **cv2.Scharr(src, ddepth, dx, dy)**

  
> **cv2.magnitude(x벡터, y벡터) : 벡터의 크기 계산 (그래디언트 크기)**  
![](magnitude.png)
> **cv2.phase(x벡터, y벡터, T/F) : 벡터의 각도 계산 (그래디언트 방향)**
![](angle.png)  
* atan : 아크탄젠트 (y/x값을 통해 세타 값 구하기)  
* T : degree 단위 / F : radian 단위

## (실습) 마스크 기반 에지 검출
레나 영상에 대해 **x축 방향과 y축 방향으로 1차 미분 후, [그래디언트 크기 > T]**인 픽셀을 에지로 검출  
> * **np.clip(fmag, 0, 255)** : 0~255를 벗어나는 값은 0과 255로 처리  
> * **np.uint8()** : float형을 unsigned int형으로 변환

### ※(소벨) 마스크 기반 에지 검출기 : 그래디언트 '크기'만 고려!※

In [1]:
import cv2
import numpy as np

def sobel_edge():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)
    
    dx = cv2.Sobel(src, cv2.CV_32F, 1, 0) # x로 편미분 (x축으로만 1차미분)
    dy = cv2.Sobel(src, cv2.CV_32F, 0, 1) # y로 편미분 (y축으로만 1차미분)
    
    fmag = cv2.magnitude(dx, dy) # 그래디언트 크기 계산
    mag = np.uint8(np.clip(fmag, 0, 255)) # 0~255를 벗어나는 값은 0과 255로 처리
    _, edge = cv2.threshold(mag, 150, 255, cv2.THRESH_BINARY) # 이진화(T:150, 최대값:255)
    
    cv2.imshow('src', src)
    cv2.imshow('mag', mag)
    cv2.imshow('edge', edge)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
sobel_edge()

![](sobel_result.png)

## 캐니 에지 검출기
* 소벨 에지 검출 방법의 단점 해결
* **좋은 에지 검출기의 조건 3가지** (만족 : 캐니 에지 검출기)
    1. 정확한 검출 : 에지를 검출하지 못하거나, 에지가 아닌 것을 에지로 검출할 확률 최소화
    2. 정확한 위치 : 실제 에지의 중심을 찾아야 함
    3. 단일 에지 : 하나의 에지는 하나의 점을 표현되어야 함 (하나의 점들의 집합)  
  
    
* **서로 '연결'되어 있는 가능성을 고려하여 그래디언트 크기 약한 에지도 놓치지 않고 찾을 수 있음!!**

### ※캐니 에지 검출기 : 그래디언트의 '크기'와 '방향' 모두 고려!※
소벨, 샤르 필터(마스크) 기반 에지 검출기 : **'크기'**만 고려 후 T 이상 값으로

* 캐니 에지 검출기의 4개의 연산 과정
    1. 가우시안 필터링 (for. 노이즈 제거)
    2. 그래디언트 계산
    3. 비최대 억제
    4. 이중 임계값을 이용한 히스테리시스 에지 트래킹

## (1) 가우시안 필터링
for 잡음 제거 (노이즈가 심하지 않으면 생략 가능)

## (2) 그래디언트 계산
* 보통 3x3 소벨 마스크 사용하여 그래디언트 계산
* 가로, 세로 방향으로 각각 소벨 마스크 필터링 수행 후 크기, 방향 모두 계산
* L2놈 or L1놈 형태로 계산 (디폴트 : L1)
    * || f || = |fx| + |fy|

## (3) 비최대 억제 (non-maximum suppression)
* 최대가 아닌 것은 억제하는 것
* 에지 근방의, 에지가 아닌 여러 픽셀들이 한꺼번에 에지로 선택되는 것을 방지! (두껍지 않게)
* 그래디언트 벡터와 **같은 방향에 있는 인접 픽셀끼리만 국지적 최대 검사 수행**
    * 결과적으로 **가장 변화율이 큰** 위치의 픽셀만 에지로 검색됨

## (4) 이중 임계값을 이용한 히스테리시스 에지 트래킹
* 하나의 T(임계값)을 사용할 경우 환경변화에 민감해지는 것을 보완 
> 따라서 **2개의 임계값** 사용!  
* 픽셀값 > T high : 강한 에지
* 픽셀값 < T low : 에지가 아님
* T low < 픽셀값 < T high : **약한 에지**
    * **Tlow와 Thigh 사이**인 픽셀은 추가적인 검사 수행 ==> **히스테리시스 에지 트래킹!**  
> if) 약한 에지 & 강한 에지 픽셀과 서로 **연결**되어 있다 ==> **에지**로 판단  
> if) 강한 에지와 **연결되어 있지 않은** 약한 에지 ==> **에지가 아니다**로 판단 

![](canny_edge(4).png)
(a) : 모든 약한 에지 부분이 강한 에지와 연결되어 있으므로 모두 에지로 판단  
(b) : 강한 에지와 연결된 부분이 없으므로 전부 에지가 아님
(c) : 강한 에지와 에지가 아닌 부분 모두 연결되어있는 부분은, 강한 에지와 연결되어있는 부분까지만 에지로 판단

## (실습) 캐니 에지 검출기
* (50, 100) : 임계값이 비교적 낮으므로, 좀 더 많은 에지들까지도 검출됨
* (50, 150) : 임계값이 비교적 높으므로, 좀 더 적은 에지들만 검출됨

In [2]:
def canny_edge():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)
    
    dst1 = cv2.Canny(src, 50, 100) # T low = 50, T high = 100
    dst2 = cv2.Canny(src, 50, 150)
    
    cv2.imshow('src', src)
    cv2.imshow('dst1', dst1) 
    cv2.imshow('dst2', dst2)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
canny_edge()

![](canny_result.png)

<hr>

# 직선 검출과 원 검출

## 허프 변환 직선 검출 (hough transform)
* 우선 에지를 찾아내고, 에지 픽셀들이 일직선상에 배열되어 있는지 확인해야 함
* 허프 변환 : 2차원 xy좌표상의 직선의 방정식 --> **파라미터 공간(a,b)**으로 변환
> y = ax + b --> b = -xa + y

![](xyab.png)
두 점 (x0, y0), (x1, y1) 각각을 지나는 무수한 직선들 중 일치하는 직선일 때   
> ==> ab평면으로 변환했을 때의 **(a0, b0)**